In [1]:
from pathlib import Path
from model_settings import ms
import pandas as pd
import sys
import os
import numpy as np
from model_settings import ms
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import QuantLib as ql
from itertools import product
notebook_dir = str(Path().resolve())
parent_dir = str(Path().resolve().parent)
grandparent_dir = str(Path().resolve().parent.parent)
grandgrandparent_dir = str(Path().resolve().parent.parent.parent)
sys.path.append(parent_dir)
sys.path.append(grandparent_dir)
sys.path.append(grandgrandparent_dir)


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
parent_files = os.listdir(parent_dir)
storefile = [file for file in parent_files if file.endswith('.h5')][0]

In [3]:
storefile

'alphaVantage vanillas.h5'

In [4]:
while True:
    try:
        store = pd.HDFStore(os.path.join(parent_dir,storefile))
        keys = store.keys()
        contracts_keys = pd.Series([key for key in keys if key.find('hottest_contracts')!=-1])
        surface_keys = pd.Series([key for key in keys if key.find('surface')!=-1])
        raw_data_keys = pd.Series([key for key in keys if key.find('raw_data')!=-1])
        break
    except OSError:
        print(OSError)
    finally:
        store.close()

In [5]:
keys_df = pd.DataFrame({'contracts_key': contracts_keys, 'surface_key': surface_keys, 'raw_data_key': raw_data_keys})
keys_df.iloc[:10]

,contracts_key,surface_key,raw_data_key
0,/date_2024_10_09/hottest_contracts,/date_2024_10_09/surface,/date_2024_10_09/raw_data
1,/date_2024_10_08/hottest_contracts,/date_2024_10_08/surface,/date_2024_10_08/raw_data
2,/date_2024_10_07/hottest_contracts,/date_2024_10_07/surface,/date_2024_10_07/raw_data
3,/date_2024_10_04/hottest_contracts,/date_2024_10_04/surface,/date_2024_10_04/raw_data
4,/date_2024_10_03/hottest_contracts,/date_2024_10_03/surface,/date_2024_10_03/raw_data
5,/date_2024_10_02/hottest_contracts,/date_2024_10_02/surface,/date_2024_10_02/raw_data
6,/date_2024_10_01/hottest_contracts,/date_2024_10_01/surface,/date_2024_10_01/raw_data
7,/date_2024_09_30/hottest_contracts,/date_2024_09_30/surface,/date_2024_09_30/raw_data
8,/date_2024_09_27/hottest_contracts,/date_2024_09_27/surface,/date_2024_09_27/raw_data
9,/date_2024_09_26/hottest_contracts,/date_2024_09_26/surface,/date_2024_09_26/raw_data


In [6]:
for i,row in keys_df.iterrows():
    surface_key = row['surface_key']
    contracts_key = row['contracts_key']
    raw_data_key = row['raw_data_key']
    
    second_backslash_pos = surface_key.find('/', 1)
    date_key_component = surface_key[:second_backslash_pos+1]
    date = surface_key[surface_key.find('_',0)+1:surface_key.find('_',0)+11]
    
    calculation_datetime = datetime.strptime(date,'%Y_%m_%d')
    calculation_date = ql.Date(
        calculation_datetime.day,
        calculation_datetime.month,
        calculation_datetime.year
    )
    printdate = calculation_datetime.strftime('%A, %Y-%m-%d')
    """
    HDF5 collection
    """
    while True:
        try:
            store = pd.HDFStore(os.path.join(parent_dir,storefile))
            raw_data = pd.DataFrame(store[raw_data_key])
            contracts = pd.DataFrame(store[contracts_key])
            vol_matrix = pd.DataFrame(store[surface_key])
            break
        except OSError:
            print(OSError)
            print('retrying in')
            for i in range (0,5):
                print(5-i)
        finally:
            store.close()
    
    vol_matrix = vol_matrix.sort_index().drop_duplicates()
    pd.to_numeric(raw_data['last'])
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    raw_data['expiration'] = pd.to_datetime(raw_data['expiration'])
    raw_data['implied_volatility'] = pd.to_numeric(raw_data['implied_volatility']).astype(float)
    raw_data['strike'] = pd.to_numeric(raw_data['strike'])
    raw_data['last'] = pd.to_numeric(raw_data['last'])
    contract_maturities = np.array((raw_data['expiration'] - raw_data['date']) / timedelta(days=1)).astype(int)
    raw_data['days_to_maturity'] = contract_maturities
    
    s = float(contracts['spot_price'].unique()[0])
    T = vol_matrix.columns.tolist()
    K = vol_matrix.index.tolist()
    r = 0.04
    g = 0.018
    
    ql.Settings.instance().evaluationDate = calculation_date
    flat_ts, dividend_ts = ms.ql_ts_rg(r, g, calculation_date)
    S_handle = ql.QuoteHandle(ql.SimpleQuote(s))
    
    heston_helpers = []
    v0 = 0.01; kappa = 0.2; theta = 0.02; rho = -0.75; eta = 0.5;
    process = ql.HestonProcess(
        flat_ts,
        dividend_ts,
        S_handle,
        v0,                # Initial volatility
        kappa,             # Mean reversion speed
        theta,             # Long-run variance (volatility squared)
        eta,               # Volatility of the volatility
        rho                # Correlation between asset and volatility
    )
    model = ql.HestonModel(process)
    engine = ql.AnalyticHestonEngine(model)
    
    for t in T:
        for k in K:
            p = ql.Period(int(t),ql.Days)
            volatility = vol_matrix.loc[k,t]
            helper = ql.HestonModelHelper(
                p, ms.calendar, float(s), k, 
                ql.QuoteHandle(ql.SimpleQuote(volatility)), 
                flat_ts, 
                dividend_ts
                )
            helper.setPricingEngine(engine)
            heston_helpers.append(helper)
    
    lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
    
    
    model.calibrate(heston_helpers, lm,
                      ql.EndCriteria(1000, 50, 1.0e-8,1.0e-8, 1.0e-8))
    
    theta, kappa, eta, rho, v0 = model.params()
    heston_parameters = pd.Series(
        [theta, kappa, eta, rho, v0],
        index = ['theta', 'kappa', 'eta', 'rho', 'v0'],
        dtype = float
    )
    
    
    calibration_test_data = raw_data.copy()[['strike','type','last','implied_volatility','days_to_maturity']]
    calibration_test_data.columns = ['strike_price','w','market_price','volatility','days_to_maturity']
    calibration_test_data['spot_price'] = s
    calibration_test_data['risk_free_rate'] = r
    calibration_test_data['dividend_rate'] = g
    calibration_test_data = calibration_test_data[calibration_test_data['days_to_maturity'].isin(contracts['days_to_maturity'])]
    calibration_test_data = calibration_test_data[calibration_test_data['days_to_maturity'].isin(contracts['days_to_maturity'])]
    
    calibration_test_data[heston_parameters.index.tolist()] = np.tile(heston_parameters,(calibration_test_data.shape[0],1))
    calibration_test_data.loc[:,'moneyness'] = ms.vmoneyness(calibration_test_data['spot_price'].values,calibration_test_data['strike_price'].values,calibration_test_data['w'].values)
    calibration_test_data['calculation_date'] = calculation_datetime
    calibration_test_data['black_scholes'] = ms.vector_black_scholes(calibration_test_data)
    calibration_test_data['heston_price'] = ms.vector_heston_price(calibration_test_data)
    calibration_test_data.loc[:,'relative_error'] = calibration_test_data['heston_price'].values/calibration_test_data['black_scholes'].values-1
    avg = np.mean(np.abs(calibration_test_data['relative_error']))
    print(f"\n{}\n{heston_parameters}\naverage absolute relative error: {round(avg*100,4)}%")

    """
    HDF5 storage
    """
    while True:
        try:
            store = pd.HDFStore(os.path.join(parent_dir,storefile))
            store.put(
                    f"{date_key_component}calibration_test", calibration_test_data,
                    format='table'
                )
            break
        except OSError:
            print(OSError)
            print('retrying in')
            for i in range (0,5):
                print(5-i)
        finally:
            store.close()


theta    0.051864
kappa    7.126519
eta      3.146618
rho     -0.745487
v0       0.046216
dtype: float64
average absolute relative error: 17.7092%

theta    0.141165
kappa    0.527277
eta      1.291423
rho     -0.751184
v0       0.038814
dtype: float64
average absolute relative error: 22.3768%


KeyboardInterrupt: 